In [1]:
from ngraph.scenario import Scenario
from ngraph.traffic_demand import TrafficDemand
from ngraph.traffic_manager import TrafficManager
from ngraph.lib.flow_policy import FlowPolicyConfig, FlowPolicy, FlowPlacement
from ngraph.lib.algorithms.base import PathAlg, EdgeSelect
from ngraph.failure_manager import FailureManager
from ngraph.failure_policy import FailurePolicy, FailureRule, FailureCondition
from ngraph.explorer import NetworkExplorer

In [2]:
scenario_yaml = """
blueprints:
  server_pod:
    rsw:
        node_count: 48
        attrs:
          hw_component: Minipack2_128x200GE
  
  f16_2tier:
    groups:
      ssw:
        node_count: 36
        attrs:
          hw_component: Minipack2_128x200GE
      fsw:
        node_count: 96
        attrs:
          hw_component: Minipack2_128x200GE

    adjacency:
      - source: /ssw
        target: /fsw
        pattern: mesh
        link_params:
          capacity: 200
          cost: 1
          
  hgrid_2tier:
    groups:
      fauu:
        node_count: 8
        attrs:
          hw_component: Minipack2_128x200GE
      fadu:
        node_count: 36
        attrs:
          hw_component: Minipack2_128x200GE

    adjacency:
      - source: /fauu
        target: /fadu
        pattern: mesh
        link_params:
          capacity: 400
          cost: 1

  fa:
    groups:
      fa[1-16]:
        use_blueprint: hgrid_2tier
                  
  dc_fabric:
    groups:
      plane[1-8]:
        use_blueprint: f16_2tier

      pod1:
        use_blueprint: server_pod
      pod36:
        use_blueprint: server_pod
    
    adjacency:
        - source: /pod1/rsw
          target: /plane[0-9]*/fsw/fsw-1
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1
        - source: /pod36/rsw
          target: /plane[0-9]*/fsw/fsw-36
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1

  ebb:
    groups:
      eb0[1-8]:
        node_count: 4 

    adjacency:
      - source: "eb0{idx}"
        target: "eb0{idx}"
        expand_vars:
          idx: [1, 2, 3, 4, 5, 6, 7, 8]
        expansion_mode: "zip"
        pattern: "mesh"
        link_params: 
          capacity: 3200
          cost: 10
             
network:
  name: "fb_region"
  version: 1.0

  groups:
    dc[1-3, 5-6]:
      use_blueprint: dc_fabric

    fa:
        use_blueprint: fa

    ebb:
        use_blueprint: ebb

  adjacency:
        - source: ".*/ssw/"
          target: ".*/fa{fa_id}/fadu"
          expand_vars:
            fa_id: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/fauu-[15]
          target: .*/eb0[1-8]-1
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1
        - source: .*/fauu-[26]
          target: .*/eb0[1-8]-2
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1   
        - source: .*/fauu-[37]
          target: .*/eb0[1-8]-3
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1 
        - source: .*/fauu-[48]
          target: .*/eb0[1-8]-4
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1  
components:
  Minipack2_128x200GE:
    component_type: router
    power_watts: 1750  # typical power consumption with 128x200GE QSFP56 200G-FR4 at 30C
  QSFP56_200G-FR4_2km:
    component_type: pluggable_optics
    power_watts: 6.5  
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network

In [3]:
network.nodes["dc1/plane1/ssw/ssw-1"]

Node(name='dc1/plane1/ssw/ssw-1', disabled=False, attrs={'hw_component': 'Minipack2_128x200GE', 'type': 'node'})

In [4]:
comp_lib = scenario.components_library
comp_lib.get("Minipack2_128x200GE")

Component(name='Minipack2_128x200GE', component_type='router', description='', cost=0.0, power_watts=1750.0, power_watts_max=0.0, capacity=0.0, ports=0, count=1, attrs={}, children={})

In [5]:
network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/eb.*",
    mode="combine",
    shortest_path=True,
)

{('.*/fsw.*', '.*/eb.*'): 819200.0}

In [ ]:
# Profiling
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/eb.*",
    mode="combine",
    shortest_path=True,
)

profiler.disable()

# stats = pstats.Stats(profiler)
# stats.sort_stats(pstats.SortKey.TIME).print_stats()

         6577676 function calls (6562273 primitive calls) in 1.901 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   339840    0.324    0.000    1.006    0.000 /Users/networmix/ws/NetGraph/ngraph/lib/graph.py:143(add_edge)
   339840    0.223    0.000    0.240    0.000 /Users/networmix/ws/NetGraph/ngraph-venv/lib/python3.13/site-packages/networkx/classes/coreviews.py:81(__getitem__)
   339840    0.196    0.000    0.273    0.000 /Users/networmix/ws/NetGraph/ngraph-venv/lib/python3.13/site-packages/networkx/classes/multidigraph.py:417(add_edge)
        1    0.194    0.194    0.214    0.214 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/spf.py:94(_spf_fast_all_min_cost_with_cap_remaining_dijkstra)
        1    0.133    0.133    0.171    0.171 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/flow_init.py:6(init_flow_graph)
        1    0.131    0.131    0.152    0.152 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/cal

In [7]:
explorer = NetworkExplorer.explore_network(network, scenario.components_library)

In [8]:
explorer.print_tree(skip_leaves=True, detailed=False)

- root | Nodes=6016, Links=167984, Cost=0.0, Power=10472000.0
  - dc1 | Nodes=1056, Links=32256, Cost=0.0, Power=1848000.0
    - plane1 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane2 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane3 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane4 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Nodes=96, Links=3456, Cost=0.0, Power=168000.0
    - plane5 | Nodes=132, Links=4032, Cost=0.0, Power=231000.0
      - ssw | Nodes=36, Links=4032, Cost=0.0, Power=63000.0
      - fsw | Node